In [1]:
workspace()

In [4]:
type Variable
    name::AbstractString
    domain::Array
    evidence_index::Int32
    assignment_index::Int32
    
    add_values::Function
    value_index::Function
    domain_size::Function
    set_assignment::Function
    get_assignment_index::Function
    get_evidence::Function
    
    function Variable(name="", domain=[])
        """Create a variable object, specifying its name (a
        string). Optionally specify the initial domain."""
        
        this = new()
        
        this.name = name
        this.domain = copy(domain)
        
        this.evidence_index = 1
        this.assignment_index = 1
        
#         Fixed
        this.add_values = function(values::Array)
            """Add domain values to the domain. values should be a list."""
            this.domain = vcat(this.domain, value)
        end
        
#       Fixed
        this.value_index = function(value::Any)
            """Domain values need not be numbers, so return the index
           in the domain list of a variable value"""
            return findfirst(this.domain, value)
        end
        
#         Fixed
        this.domain_size =  function()
            """Return the size of the domain"""
            return size(this.domain)[1]
        end
        
#         Fixed
        this.set_assignment = function(value)
            """Set this variable's assignment value for factor lookups"""
            this.assignment_index = this.value_index(value)
        end
        
#         Fixed
#         These routines are special low-level routines used directly by the
#         actor objects
        this.get_assignment_index = function()
           return this.assignment_index 
        end
        
        
        this.get_evidence = function()
            return this.domain[this.evidence_index]
        end
        return this
    end
    
end

In [5]:
type Factor
    name::AbstractString
    scope::Array
    size::Int64
    values::Array
    
    get_value::Function
    get_scope::Function
    add_scope::Function
    add_values::Function
    add_value_at_current_assignments::Function
    get_value_at_current_assignments::Function
    
    function Factor(name="NEW", scope=[])
        """create a Factor object, specify the Factor name (a string)
        and its scope (an ORDERED list of variable objects)."""
        
        this = new()
        
        this.name = name
        this.scope = scope
        
        size = 1
        for v in scope
            size *= v.domain_size()
        end
        this.values = zeros(size)
        
        this.size = size
        
#         Проверено
        this.get_scope = function ()
            """returns copy of scope...you can modify this copy without affecting
           the factor object"""
            return this.scope
        end
        
        
        this.add_scope = function (added::Array)
            this.scope = vcat(this.scope, added)
        end
        
#         this.add = function (value::Node)
#             this.value = value
#         end
        
        
#        fixed bag with indexes
        this.add_values =  function(values::Array)
            """This routine can be used to initialize the factor. We pass
        it a list of lists. Each sublist is a ORDERED sequence of
        values, one for each variable in self.scope followed by a
        number that is the factor's value when its variables are
        assigned these values. For example, if self.scope = [A, B, C],
        and A.domain() = [1,2,3], B.domain() = ['a', 'b'], and
        C.domain() = ['heavy', 'light'], then we could pass add_values the
        following list of lists
        [[1, 'a', 'heavy', 0.25], [1, 'a', 'light', 1.90],
         [1, 'b', 'heavy', 0.50], [1, 'b', 'light', 0.80],
         [2, 'a', 'heavy', 0.75], [2, 'a', 'light', 0.45],
         [2, 'b', 'heavy', 0.99], [2, 'b', 'light', 2.25],
         [3, 'a', 'heavy', 0.90], [3, 'a', 'light', 0.111],
         [3, 'b', 'heavy', 0.01], [3, 'b', 'light', 0.1]]
         This list initializes the factor so that, e.g., its value on
         (A=2,B=b,C='light) is 2.25"""
            
            for val in values
               idx = 0
                for v in this.scope
                    idx = idx * v.domain_size() + v.value_index(val[1]) -1 
                    print(idx)
                    val = val[2:end]
                end
                idx += 1
                this.values[idx] = val[1]
            end
        end
        
        this.get_value = function(variable_values)
            """This function is used to retrieve a value from the
        factor. We pass it an ordered list of values, one for every
        variable in self.scope. It then returns the factor's value on
        that set of assignments.  For example, if self.scope = [A, B,
        C], and A.domain() = [1,2,3], B.domain() = ['a', 'b'], and
        C.domain() = ['heavy', 'light'], and we invoke this function
        on the list [1, 'b', 'heavy'] we would get a return value
        equal to the value of this factor on the assignment (A=1,
        B='b', C='light')"""
            idx = 0
            for v in this.scope
                idx = idx * v.domain_size() + v.value_index(variable_values[1]) - 1
                variable_values = variable_values[2:end]
            end
            idx += 1
            return this.values[idx]
        end
        
#       As I checked - it is work
        this.add_value_at_current_assignments = function(number)
            """This function allows adding values to the factor in a way
        that will often be more convenient. We pass it only a single
        number. It then looks at the assigned values of the variables
        in its scope and initializes the factor to have value equal to
        number on the current assignment of its variables. Hence, to
        use this function one first must set the current values of the
        variables in its scope.
        For example, if self.scope = [A, B, C],
        and A.domain() = [1,2,3], B.domain() = ['a', 'b'], and
        C.domain() = ['heavy', 'light'], and we first set an assignment for A, B
        and C:
        A.set_assignment(1)
        B.set_assignment('a')
        C.set_assignment('heavy')
        then we call
        add_value_at_current_assignment(0.33)
         with the value 0.33, we would have initialized this factor to have
        the value 0.33 on the assigments (A=1, B='1', C='heavy')
        This has the same effect as the call
        add_values([1, 'a', 'heavy', 0.33])
        One advantage of the current_assignment interface to factor values is that
        we don't have to worry about the order of the variables in the factor's
        scope. add_values on the other hand has to be given tuples of values where
        the values must be given in the same order as the variables in the factor's
        scope.
        See recursive_print_values called by print_table to see an example of
        where the current_assignment interface to the factor values comes in handy."""
            idx = 0
            
            for v in this.scope
               idx = idx * v.domain_size() + v.get_assignment_index() -1 
            end
            idx += 1
            this.values[idx] =  number
        end
        
        
        this.get_value_at_current_assignments = function()
            """This function is used to retrieve a value from the
        factor. The value retrieved is the value of the factor when
        evaluated at the current assignment to the variables in its
        scope.
        For example, if self.scope = [A, B, C], and A.domain() =
        [1,2,3], B.domain() = ['a', 'b'], and C.domain() = ['heavy',
        'light'], and we had previously invoked A.set_assignment(1),
        B.set_assignment('a') and C.set_assignment('heavy'), then this
        function would return the value of the factor on the
        assigments (A=1, B='1', C='heavy')"""
            
            idx = 0
            
            for v in this.scope
               idx = idx * v.domain_size() + v.get_assignment_index() - 1
            end
            idx += 1
            return this.values[idx]
        end
        
        
        return this
    end
end

In [ ]:
type BN
    name::AbstractString
    Variables::Array{Variables}
end

In [14]:
function restrict_factor(F, var, value)
    """F is a factor, var is a Variable, and value is a value from var.domain.
    Return a new factor that is the restriction of f by this var = value.
    Don't change f! If f has only one variable its restriction yields a
    constant factor"""
    
    newFactorName = F.name * "_{R:}" * var.name * " = " * string(value) * "}"
    newFactorScope = copy(F.get_scope())
    
    var.set_assignment(value)
    deleteat!(newFactorScope, findin(newFactorScope, [var]))
    
    newFactor = Factor(newFactorName, newFactorScope)
    
    rec_restrict_factor(F, newFactorScope, newFactor)
    
    return newFactor
end

restrict_factor (generic function with 1 method)

In [15]:
function rec_restrict_factor(oldFactor, newFactorScope, newFactor)
    if isempty(newFactorScope)
        old_assign = oldFactor.get_value_at_current_assignments()
        newFactor.add_value_at_current_assignments(old_assign)
    else
        print(newFactorScope)
        for var_val in newFactorScope[1].domain
            newFactorScope[1].set_assignment(var_val)
            rec_restrict_factor(oldFactor, newFactorScope[2:end], newFactor)
        end
    end
end

rec_restrict_factor (generic function with 1 method)

In [56]:
function step1(NetFactors, EvidenceVars)
    for i in range(1, size(NetFactors)[1])
        for e_v in EvidenceVars
            if e_v in NetFactors[i].get_scope()
                newFactor = restrict_factor(NetFactors[i], e_v, e_v.get_evidence())
                
                #replace each factor in NetFactors with it's restriction f_{E=e_v}
                NetFactors[i] = copy(newFactor)
            end
        end
    end
end

step1 (generic function with 1 method)

In [31]:
function multiply_factors(Factors)
    """return a new factor that is the product of the factors in Factors"""
    #Getting the new factor name
    newFactorName = ""
    for i in range(1, size(Factors)[1])
        cur_factor = Factors[i]
        if i == 1
            newFactorName *= cur_factor.name
            continue
        else
            newFactorName = newFactorName * "_" * cur_factor.name * "_"
            continue
        end
        
    end
    
    #Getting the new factor scope
    newFactorScope = Variable[]
    
    for factor in Factors
        for var in factor.get_scope()
            if var in newFactorScope
                continue
            end
            push!(newFactorScope, var)
        end
    end
    
    #Creating the new factor
    newFactor = Factor(newFactorName, newFactorScope)
    #Setting the current assignments in the newFactor
    rec_multiply_factors(Factors, newFactorScope, newFactor, 1)
    
    return newFactor
    
end

multiply_factors (generic function with 1 method)

In [32]:
function rec_multiply_factors(Factors, newFactorScope, newFactor, total=1)

    println(newFactorScope)
    if !isempty(newFactorScope)

        for factor in Factors

            total *= factor.get_value_at_current_assignments()

        end

        newFactor.add_value_at_current_assignments(total)
    else
        for var_val in newFactorScope[1].domain()
            newFactorScope[1].set_assignment(var_val)

            rec_multiply_factors(Factors, newFactorScope[2:end], newFactor, total)
        end
    end
end

rec_multiply_factors (generic function with 2 methods)

In [59]:
function remove_var(var, new_scope, scopes)
    """Return the new set of scopes that arise from eliminating var
    from scopes"""
    
    new_scopes = []
    for s in scopes
        if !(var in s)
             push!(new_scopes, s)
        end
    end
    
    push!(new_scopes, new_scope)
    
    return new_scopes
end

remove_var (generic function with 1 method)

In [60]:
function min_fill_var(scopes, Vars)
    """Given a set of scopes (lists of lists of variables) compute and
    return the variable with minimum fill in. That the variable that
    generates a factor of smallest scope when eliminated from the set
    of scopes. Also return the new scope generated from eliminating
    that variable."""
    minv = Vars[1]
    (minfill, min_new_scope) = compute_fill(scopes, Vars[0])
    
    for v in Vars[2:end]
        (fill, new_scope) = compute_fill(scopes, v)
        if fill < minfill
            minv = v
            minfill = fill
            
            min_new_scope = new_scope
        end
    end
    return (minv, min_new_scope)

end

min_fill_var (generic function with 1 method)

In [61]:
function compute_fill(scopes, var)
    """'Return the fill in scope generated by eliminating var from
    scopes along with the size of this new scope"""
    union = []
    for s in scopes
        if var in s
            for v in s
                if !(v in union)
                   push!(union, v) 
                end
            end
        end
    end
    if var in union
        deleteat!(union, findin(union, var))
    end
    
    return (size(union)[1], union)
end

compute_fill (generic function with 1 method)

In [62]:
function min_fill_ordering(Factors, QueryVar)
    """Compute a min fill ordering given a list of factors. Return a list
    of variables from the scopes of the factors in Factors. The QueryVar is
    NOT part of the returned ordering"""
    scopes = []
    for f in Factors
       push!(scopes, f.get_scope())
    end
    
    Vars = []
    
    for s in scopes
        for v in s
            if !(v in Vars) && (v != QueryVar)
                push!(Vars, v)
            end
        end
    
    ordering = []
    
    while !empty(Vars)
        (var, new_scope) = min_fill_var(scopes, Vars)
        push!(ordering, var)
        
        if var in Vars
            deleteat!(Vars, findin(Vars, v))
        end
        
        scopes = remove_var(var, new_scope, scopes)
    end
end

min_fill_ordering (generic function with 1 method)

In [63]:
function sum_out_variable(f, var)
    """return a new factor that is the product of the factors in Factors
       followed by the suming out of Var"""
    newFactorName = f.name + "_{S: " + var.name + "}"
    newFactorScope = f.get_scope()
    
    deleteat!(newFactorScope, findin(newFactorScope, var))
    
    newFactor = Factor(newFactor, newFactorScope)
    
    rec_sumout_vars(newFactor, f, newFactorScope, var)
    
    return newFactor
end

sum_out_variable (generic function with 1 method)

In [65]:
function rec_sumout_variable(newFactor, f, newFactorScope, variable, summ=0)
   if !(size(newFactorScope)[1])
        for val in variable.domain()
            variable.set_assignment(val)
            curr = f.get_value_at_current_assignments
            summ += curr
        end
        
        newFactor.add_value_at_current_assignments(summ)
        
    else
        for var_val in newFactorScope[1].domain()
            newFactorScope[1].set_assignment(var_val)
            rec_sumout_variable(newFactor, f, newFactorScope[2:end], variable)
        end
    end
end

rec_sumout_variable (generic function with 2 methods)

In [47]:
e = [1, 2, 3]
size(e)[1]

3

In [40]:
deleteat!(e, findin(e, 3))

2-element Array{Int64,1}:
 1
 2

In [66]:
function step2(NetFactors, QueryVar)
   for v in min_fill_ordering(NetFactors, QueryVar)
#         STEP 2.A)
# finding factors f_1, ..., f_k s.t. z_j in scope(f_i)
        f_v = []
        for factor in NetFactors
            if v in factor.get_scope()
               push!(f_v, factor) 
            end
        end
        #product of factors that include z_j
        prod_f = multiply_factors(f_v)
        
         #sumout the product over z_j
        g_j = sum_out_variable(prod_f, v)
        
         #STEP 2.B)
    #remove the factors that mention z_j
        for factor in f_v
            deleteat!(NetFactors, findin(NetFactors, factor))
        end
        
        #add new factor g_j
        push!(NetFactors, g_j)
    end
end

step2 (generic function with 1 method)

In [70]:
function step3(NetFactors)
    #take product of factors with Q
    remaining_factors_w_Q = multiply_factors(NetFactors)
    
    #get the normalization factor
    normalization_factor = 0
    
    for i in remaining_factors_w_Q.values
        normalization_factor += i
    end
    
    #check if zero division will occur
    if !(normalization_factor)
        error("ZeroDivision")
    end
    
    #get list of posterior distribution
    list_of_pos = []
    for i in remaining_factors_w_Q.values
        push!(list_of_pos, i/normalization_factor)
    end
    
    return list_of_pos
end

step3 (generic function with 1 method)

In [72]:
function VariableElimination(Net, QueryVar, EvidanceVars)
    """Input: Net---a BN object (a Bayes Net)
           QueryVar---a Variable object (the variable whose distribution
                      we want to compute)
           EvidenceVars---a LIST of Variable objects. Each of these
                          variables has had its evidence set to a particular
                          value from its domain using set_evidence.
   VE returns a distribution over the values of QueryVar, i.e., a list
   of numbers one for every value in QueryVar's domain. These numbers
   sum to one, and the i'th number is the probability that QueryVar is
   equal to its i'th value given the setting of the evidence
   variables. For example if QueryVar = A with Dom[A] = ['a', 'b',
   'c'], EvidenceVars = [B, C], and we have previously called
   B.set_evidence(1) and C.set_evidence('c'), then VE would return a
   list of three numbers. E.g. [0.5, 0.24, 0.26]. These numbers would
   mean that Pr(A='a'|B=1, C='c') = 0.5 Pr(A='a'|B=1, C='c') = 0.24
   Pr(A='a'|B=1, C='c') = 0.26"""
    NetFactors = Net.factors()
    
    step1(NetFactors, EvidanceVars)
    
    step2(NetFactors, QueryVar)
    
    return step3(NetFactors)
end

VariableElimination (generic function with 1 method)

Example below

In [7]:
V = Variable("V", [0,1 ])
G = Variable("G", [0,1])
R = Variable("R", [0,1])
S = Variable("S", [0,1])

Variable("S",[0,1],1,1,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))

In [8]:
fac_V = Factor("V", [V])
fac_G = Factor("G", [G])
fac_R = Factor("R", [V, G, R])
fac_S = Factor("S", [G, S])

Factor("S",[Variable("G",[0,1],1,1,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("S",[0,1],1,1,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))],4,[0.0,0.0,0.0,0.0],(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))

In [9]:
fac_V.add_values([(0, 0.33), (1, 0.66)])
fac_G.add_values([(0, 0.25), (1, 0.75)])
fac_R.add_values([(0, 0, 0, 0.6), (0, 0, 1, 0.4),
        (0, 1, 0, 0.3), (0, 1, 1, 0.7),
        (1,  0, 0, 0.2), (1, 0, 1, 0.8),
        (1, 1, 0, 0.5), (1, 1, 1, 0.5)])
fac_S.add_values([(0, 0, 0.2), (0, 1, 0.8),
        (1, 0, 0.5), (1, 1, 0.5)])

010100000101201312412513613700011213

In [10]:
V.assignment_index

1

In [45]:
fac_V.get_value_at_current_assignments()

0.66

In [16]:
newF = restrict_factor(fac_R, V, 1)

[Variable("G",[0,1],1,1,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("R",[0,1],1,1,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))][Variable("R",[0,1],1,1,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))][Variable("R",[0,1],1,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))]

Factor("R_{R:}V = 1}",[Variable("G",[0,1],1,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("R",[0,1],1,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))],4,[0.2,0.8,0.5,0.5],(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))

In [38]:
fac_V, fac_G

(Factor("V",[Variable("V",[0,1],1,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))],2,[0.33,0.66],(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Factor("G",[Variable("G",[0,1],1,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))],2,[0.25,0.75],(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)))

In [39]:
mul_f = multiply_factors([fac_G, fac_V])

[Variable("G",[0,1],1,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("V",[0,1],1,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))]


Factor("G_V_",[Variable("G",[0,1],1,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("V",[0,1],1,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))],4,[0.0,0.0,0.0,0.495],(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))

In [46]:
mul_f.get_value(

LoadError: wrong number of arguments

In [ ]:
restrict_factor(fac_R, V, 1).get_value([0, 0])